# Simpsons characters dialogs

In [1]:
import pandas as pd
from flask import Flask, request

In [2]:
df = pd.read_csv('input/simpsons_dataset.csv', engine = 'python')
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
nombres = { 
    'raw_character_text':'Characters',
    'spoken_works':'Text'
}
df = df.rename(columns=nombres)

# Create characters groups

In [38]:
df = df.dropna()
df = df[(df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson', 'Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince', 'Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson')))]
df

,Characters,spoken_words,Groups
1,Lisa Simpson,Where's Mr. Bergstrom?,Home
3,Lisa Simpson,That life is worth living.,Home
5,Martin Prince,I don't think there's anything left to say.,School
7,Bart Simpson,Victory party under the slide!,Home
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!,Home
...,...,...,...
158303,Lisa Simpson,Does Bart have to be there?,Home
158304,Marge Simpson,Yes.,Home
158305,Lisa Simpson,Can we do it this week?,Home
158307,Lisa Simpson,"Mr. Bergstrom, we request the pleasure of your...",Home


In [39]:
df['Groups'] = df['Groups'][df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson'))] = 'Home'
df['Groups'][df['Characters'].isin(('Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince'))] = 'School'
df['Groups'][df['Characters'].isin(('Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson'))] = 'Bar'

In [42]:
df_reduced = df.head(100)
df_reduced = df_reduced.dropna(axis=0)
df_reduced.isnull().sum()

Characters      0
spoken_words    0
Groups          0
dtype: int64

# Including Simpsons characters on MongoDB

In [43]:
import requests

In [44]:
for character in pd.unique(df_reduced['Characters']):
    data = requests.get(f"http://localhost:8800/user/create/{character}").content
print(data)

b'{\n  "user_id": "5e51788d36a4e6707c71aa3d"\n}\n'


# Creating chats

In [45]:
for group in pd.unique(df_reduced['Groups']):
    data = requests.get(f"http://localhost:8800/chat/create/{group}").content
print(data)

b'{\n  "Group": "Bar"\n}\n'


# Adding characters to chat

In [ ]:
for character in pd.unique(df_reduced['Characters']):
    data = requests.get(f"http://localhost:8800/chat/adduser/{character}").content
print(data)

# Adding messages to chat

In [46]:
'''
Characters from the same group will be on the same MongoDB document conversation
'''
for index, row in df_reduced.iterrows():  
    data = requests.get(f"http://localhost:8800/chat/{row['Groups']}/{row['Characters']}/{row['spoken_words']}").content
print(data)

b'Great. Included'


# Pruebas

In [47]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps

In [48]:
client = MongoClient("mongodb://localhost/Conversations")
db = client.get_database()


In [ ]:
chatname = ['cole', 'casa']
for c in chatname:
    a = list(db['Conversations'].find({'Group':f'{c}'}, {'_id':1}))
print(a[0]['_id'])

In [ ]:
for username in pd.unique(dfschool['Characters']):
    b = list(db['Users'].find({'username':f'{username}'}, {'_id':1}))[0]['_id']
    print(b)

In [ ]:
addUser('casa', 'lisa')

In [80]:
#Añadir a Character
for user in pd.unique(df_reduced['Characters']):
    print(user)
    grupo = list(df_reduced['Groups'][df_reduced['Characters'] == user])[0]
    print(type(grupo))
    i = list(db['Users'].find({'username': user}, {'_id':1}))[0]['_id']
    print(i)
    ig = list(db['Conversations'].find({'Group':grupo}, {'_id':1}))[0]['_id']
    print(ig)
    query = list(db['Conversations'].update({"_id":ig}, {'$push': {'Characters': {'username': user, '_id':i}}}))
    print('User added to the chat')
    
    '''
    query = list(db['Conversations'].update())
    print(i)
    
    
    if db['Conversations'].count_documents({'Group':chatname}, limit = 1) != 0:
        i = list(db['Conversations'].find({'Group':chatname}, {'_id':1}))[0]['_id']
        query = list(db['Conversations'].update({"_id":i},{"$push":{"Message": {'username': username, 'message': message}}}))
        return 'Great. Included'
'''

Lisa Simpson
<class 'str'>
5e51691085d1a8d1ba848e85
5e5169be85d1a8d1ba848e90
User added to the chat
Martin Prince
<class 'str'>
5e51688985d1a8d1ba848e80
5e5169be85d1a8d1ba848e91
User added to the chat
Bart Simpson
<class 'str'>
5e51691085d1a8d1ba848e87
5e5169be85d1a8d1ba848e90
User added to the chat
Nelson Muntz
<class 'str'>
5e51688985d1a8d1ba848e81
5e5169be85d1a8d1ba848e91
User added to the chat
Milhouse Van Houten
<class 'str'>
5e51688985d1a8d1ba848e82
5e5169be85d1a8d1ba848e91
User added to the chat
Homer Simpson
<class 'str'>
5e51691085d1a8d1ba848e8e
5e5169be85d1a8d1ba848e90
User added to the chat
Marge Simpson
<class 'str'>
5e51691085d1a8d1ba848e8f
5e5169be85d1a8d1ba848e90
User added to the chat
Moe Szyslak
<class 'str'>
5e51788d36a4e6707c71aa3d
5e51789236a4e6707c71aa3e
User added to the chat


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  # Remove the CWD from sys.path while we load stuff.


In [79]:
for user in pd.unique(df_reduced['Characters']):
    print(user)
    grupo = list(df_reduced['Groups'][df_reduced['Characters'] == user])[0]
    print(grupo)
    i = list(db['Users'].find({'username': user}, {'_id':1}))[0]['_id']
    print(i)
    ig = list(db['Conversations'].find({'Group':grupo}, {'_id':1}))[0]['_id']
    print(ig)

Lisa Simpson
Home
5e51691085d1a8d1ba848e85
5e5169be85d1a8d1ba848e90
Martin Prince
School
5e51688985d1a8d1ba848e80
5e5169be85d1a8d1ba848e91
Bart Simpson
Home
5e51691085d1a8d1ba848e87
5e5169be85d1a8d1ba848e90
Nelson Muntz
School
5e51688985d1a8d1ba848e81
5e5169be85d1a8d1ba848e91
Milhouse Van Houten
School
5e51688985d1a8d1ba848e82
5e5169be85d1a8d1ba848e91
Homer Simpson
Home
5e51691085d1a8d1ba848e8e
5e5169be85d1a8d1ba848e90
Marge Simpson
Home
5e51691085d1a8d1ba848e8f
5e5169be85d1a8d1ba848e90
Moe Szyslak
Bar
5e51788d36a4e6707c71aa3d
5e51789236a4e6707c71aa3e
